# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 13 2022 9:25AM,248530,10,0086073172,ISDIN Corporation,Released
1,Oct 13 2022 9:25AM,248530,10,0086073171,ISDIN Corporation,Released
2,Oct 13 2022 9:25AM,248530,10,0086073173,ISDIN Corporation,Released
3,Oct 13 2022 9:25AM,248530,10,0086073180,ISDIN Corporation,Released
4,Oct 13 2022 9:25AM,248530,10,0086073179,ISDIN Corporation,Released
5,Oct 13 2022 9:25AM,248530,10,0086073178,ISDIN Corporation,Released
6,Oct 13 2022 9:25AM,248530,10,0086073184,ISDIN Corporation,Released
7,Oct 13 2022 9:25AM,248530,10,0086073186,ISDIN Corporation,Released
8,Oct 13 2022 9:25AM,248530,10,0086073185,ISDIN Corporation,Released
9,Oct 13 2022 9:25AM,248530,10,0086073190,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
29,248527,Executing,6
30,248527,Released,5
31,248528,Released,1
32,248529,Released,15
33,248530,Released,49


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
248526,NaN,NaN,2.0
248527,NaN,6.0,5.0
248528,NaN,NaN,1.0
248529,NaN,NaN,15.0
248530,NaN,NaN,49.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248081,0.0,1.0,0.0
248083,0.0,0.0,1.0
248087,0.0,0.0,1.0
248089,0.0,0.0,1.0
248091,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248081,0,1,0
248083,0,0,1
248087,0,0,1
248089,0,0,1
248091,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248081,0,1,0
1,248083,0,0,1
2,248087,0,0,1
3,248089,0,0,1
4,248091,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248081,,1,
1,248083,,,1
2,248087,,,1
3,248089,,,1
4,248091,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 13 2022 9:25AM,248530,10,ISDIN Corporation
49,Oct 13 2022 9:11AM,248529,10,ISDIN Corporation
64,Oct 13 2022 8:56AM,248528,19,"Zymergen, Inc."
65,Oct 13 2022 8:52AM,248527,12,Hush Hush
76,Oct 13 2022 8:42AM,248526,19,ACG North America LLC
78,Oct 13 2022 8:30AM,248514,10,ISDIN Corporation
111,Oct 13 2022 8:23AM,248525,19,ACG North America LLC
112,Oct 13 2022 7:46AM,248522,19,"AdvaGen Pharma, Ltd"
118,Oct 12 2022 4:56PM,248521,21,"NBTY Global, Inc."
119,Oct 12 2022 4:43PM,248091,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 13 2022 9:25AM,248530,10,ISDIN Corporation,,,49
1,Oct 13 2022 9:11AM,248529,10,ISDIN Corporation,,,15
2,Oct 13 2022 8:56AM,248528,19,"Zymergen, Inc.",,,1
3,Oct 13 2022 8:52AM,248527,12,Hush Hush,,6,5
4,Oct 13 2022 8:42AM,248526,19,ACG North America LLC,,,2
5,Oct 13 2022 8:30AM,248514,10,ISDIN Corporation,,,33
6,Oct 13 2022 8:23AM,248525,19,ACG North America LLC,,,1
7,Oct 13 2022 7:46AM,248522,19,"AdvaGen Pharma, Ltd",,6,
8,Oct 12 2022 4:56PM,248521,21,"NBTY Global, Inc.",,,1
9,Oct 12 2022 4:43PM,248091,21,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 13 2022 9:25AM,248530,10,ISDIN Corporation,49,,
1,Oct 13 2022 9:11AM,248529,10,ISDIN Corporation,15,,
2,Oct 13 2022 8:56AM,248528,19,"Zymergen, Inc.",1,,
3,Oct 13 2022 8:52AM,248527,12,Hush Hush,5,6,
4,Oct 13 2022 8:42AM,248526,19,ACG North America LLC,2,,
5,Oct 13 2022 8:30AM,248514,10,ISDIN Corporation,33,,
6,Oct 13 2022 8:23AM,248525,19,ACG North America LLC,1,,
7,Oct 13 2022 7:46AM,248522,19,"AdvaGen Pharma, Ltd",,6,
8,Oct 12 2022 4:56PM,248521,21,"NBTY Global, Inc.",1,,
9,Oct 12 2022 4:43PM,248091,21,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 13 2022 9:25AM,248530,10,ISDIN Corporation,49,,
1,Oct 13 2022 9:11AM,248529,10,ISDIN Corporation,15,,
2,Oct 13 2022 8:56AM,248528,19,"Zymergen, Inc.",1,,
3,Oct 13 2022 8:52AM,248527,12,Hush Hush,5,6,
4,Oct 13 2022 8:42AM,248526,19,ACG North America LLC,2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 13 2022 9:25AM,248530,10,ISDIN Corporation,49.0,NaN,NaN
1,Oct 13 2022 9:11AM,248529,10,ISDIN Corporation,15.0,NaN,NaN
2,Oct 13 2022 8:56AM,248528,19,"Zymergen, Inc.",1.0,NaN,NaN
3,Oct 13 2022 8:52AM,248527,12,Hush Hush,5.0,6.0,NaN
4,Oct 13 2022 8:42AM,248526,19,ACG North America LLC,2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 13 2022 9:25AM,248530,10,ISDIN Corporation,49.0,0.0,0.0
1,Oct 13 2022 9:11AM,248529,10,ISDIN Corporation,15.0,0.0,0.0
2,Oct 13 2022 8:56AM,248528,19,"Zymergen, Inc.",1.0,0.0,0.0
3,Oct 13 2022 8:52AM,248527,12,Hush Hush,5.0,6.0,0.0
4,Oct 13 2022 8:42AM,248526,19,ACG North America LLC,2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2236371,123.0,6.0,25.0
12,248527,5.0,6.0,0.0
15,248265,2.0,0.0,28.0
18,248462,1.0,0.0,0.0
19,1988043,11.0,9.0,2.0
21,1488952,5.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2236371,123.0,6.0,25.0
1,12,248527,5.0,6.0,0.0
2,15,248265,2.0,0.0,28.0
3,18,248462,1.0,0.0,0.0
4,19,1988043,11.0,9.0,2.0
5,21,1488952,5.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,123.0,6.0,25.0
1,12,5.0,6.0,0.0
2,15,2.0,0.0,28.0
3,18,1.0,0.0,0.0
4,19,11.0,9.0,2.0
5,21,5.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,123.0
1,12,Released,5.0
2,15,Released,2.0
3,18,Released,1.0
4,19,Released,11.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,18,19,21
Status,,,,,,
Completed,25.0,0.0,28.0,0.0,2.0,0.0
Executing,6.0,6.0,0.0,0.0,9.0,1.0
Released,123.0,5.0,2.0,1.0,11.0,5.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,18,19,21
0,Completed,25.0,0.0,28.0,0.0,2.0,0.0
1,Executing,6.0,6.0,0.0,0.0,9.0,1.0
2,Released,123.0,5.0,2.0,1.0,11.0,5.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,18,19,21
0,Completed,25.0,0.0,28.0,0.0,2.0,0.0
1,Executing,6.0,6.0,0.0,0.0,9.0,1.0
2,Released,123.0,5.0,2.0,1.0,11.0,5.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()